# C2

## 1.Decode in LLM

In [1]:
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

In [2]:
# 添加加速库
!pip install -U hf_xet

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [3]:
# !pip install bitsandbytes

In [4]:
#pip install -U bitsandbytes

In [5]:
from transformers import BitsAndBytesConfig, AutoModelForCausalLM
import torch
# 修改为半精度
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,            # 或 load_in_8bit=True
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)
model_DS = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",            # 让库自动把层拆分到 CPU/GPU
    trust_remote_code=True
)

C:\Users\hhm18\miniconda3\envs\env_LLM\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
import torch
print(torch.cuda.get_device_properties(0).total_memory / 1024**3, "GB")   # GPU 总容量

7.99560546875 GB


In [7]:
print(model_DS)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear4bit(in_features=3584, out_features=3584, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear4bit(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((3584,), 

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.special_tokens_map

{'bos_token': '<｜begin▁of▁sentence｜>',
 'eos_token': '<｜end▁of▁sentence｜>',
 'pad_token': '<｜end▁of▁sentence｜>'}

In [9]:
prompt = '帮我写一份申请书，申请steam游戏退款。'
inputs = tokenizer(prompt, return_tensors='pt').to(model_DS.device)

inputs

{'input_ids': tensor([[151646, 108965,  61443, 104191, 101915,  90286,   3837, 101915,  46590,
          99329, 113794,   1773]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [10]:
generation_output = model_DS.generate(
    input_ids = inputs.input_ids,
    max_new_tokens = 128, 
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
C:\Users\hhm18\miniconda3\envs\env_LLM\lib\site-packages\transformers\integrations\sdpa_attention.py:89: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [11]:
print(tokenizer.decode(generation_output[0],
                       skip_special_tokens=True))

帮我写一份申请书，申请steam游戏退款。我的情况是，我在购买了游戏后，发现这个游戏是中文版，而原版是英文的，导致我无法理解游戏的内容，影响了游戏体验。所以，我想申请退款。

我需要这份申请书的模板，最好能用中文写，结构清晰，容易填写。然后，我还要写一份致 steam 官方的感谢信，表达我的感谢和请求。

最后，如果可能的话，提供一个英文版的游戏退款申请模板，方便我用英文发送给国外的客服。如果有，就写在最后面。如果有其他建议，也可以提出来。

好的，现在我需要


In [12]:
inputs.input_ids

tensor([[151646, 108965,  61443, 104191, 101915,  90286,   3837, 101915,  46590,
          99329, 113794,   1773]], device='cuda:0')

In [13]:
for id_ in inputs.input_ids[0]:
    print(tokenizer.decode(id_))

<｜begin▁of▁sentence｜>
帮我
写
一份
申请
书
，
申请
steam
游戏
退款
。


In [14]:
prompt2 = 'Trip data is published monthly on this website, typically with a two-month delay to allow time for full vendor submissions.'
input_ids = tokenizer(prompt2, return_tensors='pt').input_ids.to('cuda')
for id_ in input_ids[0]:
    print(tokenizer.decode(id_))

<｜begin▁of▁sentence｜>
Trip
 data
 is
 published
 monthly
 on
 this
 website
,
 typically
 with
 a
 two
-month
 delay
 to
 allow
 time
 for
 full
 vendor
 submissions
.


不同的模型分词的结果也会有所不同

# 2Embeding 部分

## 2.1上下文相关的Embd

In [15]:
from transformers import AutoTokenizer, AutoModel

In [16]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')
prompt = 'Hello World! I love U'
tokens = tokenizer(prompt, return_tensors='pt')
output = model(**tokens)[0]

In [17]:
output.shape

torch.Size([1, 8, 768])

In [18]:
tokens

{'input_ids': tensor([[ 101, 7592, 2088,  999, 1045, 2293, 1057,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [19]:
for token in tokens['input_ids'][0]:
    print(tokenizer.decode(token))

[CLS]
hello
world
!
i
love
u
[SEP]


## 2.2文本向量（句向量）

In [20]:
!pip install sentence-transformers

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [21]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer(
    'sentence-transformers/all-distilroberta-v1'
)

In [22]:
vector_hello = model.encode('Hello world!')

In [23]:
vector_hello.shape

(768,)

In [24]:
vector_hello

array([ 1.69184674e-02, -3.36312056e-02, -3.10999062e-02, -4.66779470e-02,
        4.25718687e-02, -4.39484529e-02,  6.37702225e-03, -1.34819616e-02,
        3.60154291e-03,  7.28878751e-03,  6.12260425e-04,  3.17114182e-02,
        2.55764928e-02, -3.38461362e-02, -3.70591581e-02, -1.08320937e-02,
       -2.09056325e-02, -5.27956523e-02, -5.32541536e-02, -3.20068337e-02,
        1.11168891e-03,  7.08929636e-03, -2.89082900e-02, -4.09262400e-04,
        5.24733216e-02,  7.05229864e-02, -5.47245890e-03, -5.43525927e-02,
        4.72666658e-02,  3.61847207e-02,  2.13295799e-02, -5.09397574e-02,
       -4.36914563e-02, -2.49673482e-02, -6.53974265e-02, -1.34584950e-02,
        1.49979591e-02, -3.51658650e-02,  5.61910197e-02,  2.04538535e-02,
        1.92342885e-02, -2.02605650e-02,  3.84667069e-02,  1.69377942e-02,
       -2.73476709e-02, -2.15194263e-02, -2.46427488e-02, -8.07106495e-02,
       -5.95961465e-03,  6.77485168e-02, -1.43193472e-02, -6.93320669e-03,
       -3.91897187e-02,  

# C3.深入TransformerLLM
* tansformer LLM不同输入和输出的区别
* 不同的output可以有什么用
* 了解RMSNorm, 和layerNorm的区别
* 了解KVcache的原理，以及在推理的时候怎么使用